In [ ]:
using Pkg
# create a separate environment for the notebook
Pkg.activate(".")
Pkg.status()

In [ ]:
# add QInchworm to notebook environment
# Pkg.develop(path="../../")

In [ ]:
using Printf

In [ ]:
# assume these are in base env
using PyPlot, Revise

In [ ]:
using QInchworm
import QInchworm.diagrammatics

In [ ]:
function make_arc(x1, x2; kwargs...)
    width = x2 - x1
    height = width
    arc = matplotlib.patches.Arc((x2 - width/2, 0.0), width, height, theta2=180.0; kwargs...)
    return arc
end

function plot_topology!(ax, vertices, t::diagrammatics.Topology, k::Int = 0)
    connected, disconnected = diagrammatics.split_k_connected(t.pairs, k)
    diagrammatics.traverse_crossing_graph_dfs!(connected, disconnected)

    for p in connected
      arc = make_arc(vertices[p.first], vertices[p.second]; edgecolor="black")
      ax.add_patch(arc)
    end

    for p in disconnected
      arc = make_arc(vertices[p.first], vertices[p.second]; edgecolor="red")
      ax.add_patch(arc)
    end
end


In [ ]:
function plot_all_topologies(order::Int)
    vertices = range(0.0, 1.0, 2*order)
    t_inch = (vertices[1] + vertices[2]) / 2 # k = 1
    
    topos = diagrammatics.Topology.(diagrammatics.pair_partitions(order))
    ntopos = length(topos)
    
    nrows = Int(ceil(ntopos // 5))
    fig = plt.figure(figsize=(10, 1*nrows))
    gs = matplotlib.gridspec.GridSpec(nrows, 5, figure=fig, wspace=0, hspace=0)

    for (k, (i,j)) in zip(1:ntopos, Iterators.product(1:nrows, 1:5))
        ax = fig.add_subplot(gs[i,j])
        ax.plot(vertices, zero(vertices), "k.")

        top = topos[k]

        plot_topology!(ax, vertices, top, 1)
        ax.set_ylim([0.0, 0.55])
        ax.set(xticks=[], yticks=[])
        ax.axvline([t_inch], color="k", linestyle="--")
    end
end

In [ ]:
function plot_k_connected_topologies(order::Int, k::Int)
    vertices = range(0.0, 1.0, 2*order)
    t_inch = (vertices[k] + vertices[k+1]) / 2
    
    topos = diagrammatics.Topology.(diagrammatics.pair_partitions(order))
    
    filter!(topos) do top
        diagrammatics.is_k_connected(top, k)
    end
    
    
    ntopos = length(topos)

    nrows = Int(ceil(ntopos // 5))
    fig = plt.figure(figsize=(10, 1*nrows))
    gs = matplotlib.gridspec.GridSpec(nrows, 5, figure=fig, wspace=0, hspace=0)

    for (k, (i,j)) in zip(1:ntopos, Iterators.product(1:nrows, 1:5))
        ax = fig.add_subplot(gs[i,j])
        ax.plot(vertices, zero(vertices), "k.")

        top = topos[k]

        plot_topology!(ax, vertices, top, 1)
        ax.set_ylim([0.0, 0.55])
        ax.set(xticks=[], yticks=[])
        ax.axvline([t_inch], color="k", linestyle="--")
    end    
end

In [ ]:
plot_all_topologies(1)

In [ ]:
plot_all_topologies(2)

In [ ]:
plot_all_topologies(3)

In [ ]:
plot_all_topologies(4)

In [ ]:
plot_k_connected_topologies(1, 1)

In [ ]:
plot_k_connected_topologies(2, 1)

In [ ]:
plot_k_connected_topologies(3, 1)

In [ ]:
plot_k_connected_topologies(4, 1)

In [ ]:
orders = 1:8
ntopos_by_order = map(orders) do order
    ntopos = length(filter!(top -> diagrammatics.is_k_connected(top, 1), diagrammatics.Topology.(diagrammatics.pair_partitions(order))))
    return ntopos
end

In [ ]:
@printf("%-8s %15s %15s\n", "order", "1-connected", "all")
for (n, m, o) in zip(ntopos_by_order, map(o -> diagrammatics.double_factorial(2o - 1), orders), orders)
    @printf("%-8d %15d %15d\n", o, n, m)
     # println("order $o: ")
end

In [ ]:
fig, ax = plt.subplots()
ax.plot(orders, ntopos_by_order, label="1-connected")
ax.plot(orders, map(o -> diagrammatics.double_factorial(2o - 1), orders), label="all")
ax.set_xlabel("order")
ax.set_ylabel("number of topologies")
ax.set_yscale("log")
ax.legend()